# 비정형 데이터 분류

자연어 처리 또는 텍스트 분석을 위한 50,000개의 영화 리뷰가 있는 IMDB 데이터 셋

> https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

## #01. 준비작업

### [1] 패키지 참조

In [64]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(work_path)

from helper.util import *
from helper.plot import *
from helper.analysis import *
from helper.classification import *

from sklearn.feature_extraction.text import CountVectorizer
import contractions

### [2] 데이터 가져오기

> 데이터 파일이 약 25M 용량이므로 로딩하는데 시간이 오래 소요된다.

| 변수 | 설명 |
|---|---|
| sentiment | 긍정(`positive`), 부정(`negative`) |
| review | 리뷰내용 |

In [2]:
origin = my_read_excel("https://data.hossam.kr/mldata/imdb.xlsx", categories=['sentiment'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49998 entries, 0 to 49997
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   sentiment  49998 non-null  category
 1   review     49998 non-null  object  
dtypes: category(1), object(1)
memory usage: 439.7+ KB
None

데이터프레임 상위 5개 행
+----+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## #02. 문서 단어 행렬(Document-Term Matrix)의 이해

> 딥러닝에서는 텍스트 토큰화라고 부름.

### [1] 예시 문장

In [3]:
doc1 = "She likes python"
doc2 = "She hates python"

위 문장을 문서 단어 행렬로 표현하면 아래와 같다.

| | She | likes | hates | python |
|---|---|---|---|---|
| doc1 | 1 | 1 | 0 | 1 |
| doc2 | 1 | 0 | 1 | 1 |

즉, 샘플 문장 리스트에서 각 단어가 몇 번씩 등장했는지를 카운트 해서 행렬(=2차 배열)로 표현하는 것.

이렇게 하면 하나의 문장인 `doc1`은 `[1, 1, 0, 1]`이라는 1차원 리스트로 표현되기 때문에 수치적인 연산이 가능해진다.

### [2] 단어 행렬 만들기

`fit_transform()` 메서드는 희소 행렬을 반환하도록 되어 있다. 

이에 대해 `toarray()` 메서드를 호출하면 넘파이 배열로 바꿀 수 있다.

In [4]:
count_vec = CountVectorizer()
fit = count_vec.fit_transform([doc1, doc2])
doc_term_matrix = fit.toarray()
doc_term_matrix

array([[0, 1, 1, 1],
       [1, 0, 1, 1]], dtype=int64)

### [3] 단어 행렬 방식의 단점

In [5]:
doc1 = "Potter likes Singer"
doc2 = "Singer likes Potter"

| | Potter | likes | Singer |
|---|---|---|---|
| doc1 | 1 | 1 | 1 |
| doc2 | 1 | 1 | 1 |

의미가 다름에도 단어 행렬은 두 문장을 같은 것처럼 표현함.

단어의 순서를 고려하지 않은 분석 방법.

In [6]:
count_vec = CountVectorizer()
fit = count_vec.fit_transform([doc1, doc2])
doc_term_matrix = fit.toarray()
doc_term_matrix

array([[1, 1, 1],
       [1, 1, 1]], dtype=int64)

### [4] 서로 길이가 다른 문장의 경우

In [7]:
doc1 = "Python is very good"
doc2 = "Life is too short, You need Python"

In [8]:
count_vec = CountVectorizer()
fit = count_vec.fit_transform([doc1, doc2])
doc_term_matrix = fit.toarray()
doc_term_matrix

array([[1, 1, 0, 0, 1, 0, 0, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 1]], dtype=int64)

In [9]:
words = count_vec.get_feature_names_out()
words

array(['good', 'is', 'life', 'need', 'python', 'short', 'too', 'very',
       'you'], dtype=object)

In [10]:
DataFrame(doc_term_matrix, columns=words)

,good,is,life,need,python,short,too,very,you
0,1,1,0,0,1,0,0,1,0
1,0,1,1,1,1,1,1,0,1


> 주어진 문장에 사용되는 모든 단어의 수를 길이로 갖는 이진 리스트 형태의 데이터로 각 문장이 변환되므로 실제 데이터크기는 원래의 데이터보다 더 커진다.

### [5] 단어 순서 확인하기

#### (1) 기본 단어 행렬 생성

In [42]:
doc1 = "She likes python"
doc2 = "She hates python"

count_vec = CountVectorizer()
fit = count_vec.fit_transform([doc1, doc2])

doc_term_matrix = fit.toarray()
doc_term_matrix

array([[0, 1, 1, 1],
       [1, 0, 1, 1]], dtype=int64)

#### (2) 각 단어별 인덱스 확인

In [45]:
count_vec.vocabulary_

{'she': 3, 'likes': 1, 'python': 2, 'hates': 0}

> hates가 문서 단어의 첫 번째 열에 해당함을 알 수 있다.

#### (3) 인덱스 순서로 단어를 정렬해서 리턴

In [13]:
count_vec.get_feature_names_out()

array(['hates', 'likes', 'python', 'she'], dtype=object)

### [6] 단어 사전 구축

단어 행렬을 생성하기 위해서는 어떤 단어가 몇 번째 인덱스에 올 것인지 정해야 한다.

이를 단어 사전이라 부른다.

기본적으로 머신러닝이 문장을 단어 행렬로 변환(transform)하기 전에 fit 단계에서 이미 단어 사전을 구축한다.

분석가가 분석 목적에 맞는 사전을 미리 정의할 수 있다.

#### (1) 분석가의 사전 정의

사전에 포함되지 않은 단어가 문장에 포함될 경우 무조건 `0`번으로 분류하기 때문에 사전의 첫 번째 단어는 불용어를 의미하는 값으로 정하는 것이 좋다.

In [58]:
my_vocabulary = {'ovv': 0, 'java': 1, 'html': 2, 'css': 3, 'javascript': 4, 'php': 5, 'sql': 6, 'good': 7}

#### (2) 사전을 품은 변환객체 생성

In [61]:
count_vec = CountVectorizer(
    vocabulary=my_vocabulary,       # 사전 연결
    stop_words= ['is','to','my'],   # 불용어(금지어) 추가 설정
    lowercase=True         # 학습에 사용되는 문장을 소문자로 변환함(기본값=False)
)

# 직접 사전을 구축한 경우 프로퍼티 이름에서 언더바가 제외됨
count_vec.vocabulary

{'ovv': 0,
 'java': 1,
 'html': 2,
 'css': 3,
 'javascript': 4,
 'php': 5,
 'sql': 6,
 'good': 7}

In [63]:
fit = count_vec.fit_transform([
    "I can do Java, HTML, CSS, Javascript, PHP, Python", 
    "Python is good"
])

doc_term_matrix = fit.toarray()
doc_term_matrix

array([[0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]], dtype=int64)

> 사전에 포함되지 않은 문장은 버린다.(`0`으로 감별함)

## 03. 영어 축약형 표현의 처리 

In [65]:
contractions.fix("you're happy now")

'you are happy now'

In [66]:
contractions.fix("yall're happy now")

'you all are happy now'

In [67]:
contractions.fix("OMG")

'OMG'

In [68]:
contractions.add('OMG', 'Oh my god')
contractions.fix("OMG")

'OH MY GOD'